In [1]:
import numpy as np

def CC_DailyFlow_Historical_Northcoast(BT_MonthlyFlow, HydroCond_Ann, Precip, AnalogYr, BTDFlows, WY, SC_Climate):
    
    # Get the length for use below
    HydroLength = len(BT_MonthlyFlow)

    # Historical Laguna daily flows
    LGDFlow = np.loadtxt('./Input/LGxl_Catalog.txt', delimiter = ',')
    Fidx = np.where((LGDFlow[:,0]==9) & (LGDFlow[:,1]==30) & (LGDFlow[:,2]==1938))[0][0] #index of Sept 30, 1938
    A = LGDFlow[:Fidx+1,:]
    LGDFlows = np.concatenate((A, LGDFlow), axis=0)
    LGDFlows[:Fidx+1,:3] = BTDFlows[:Fidx+1,:3]
    
    # Historical Laguna daily flows
    MJDFlow = np.loadtxt('./Input/MJxl_Catalog.txt', delimiter = ',')
    B = MJDFlow[:Fidx+1,:]
    MJDFlows = np.concatenate((B, MJDFlow), axis=0)
    MJDFlows[:Fidx+1,:3] = BTDFlows[:Fidx+1,:3]
    
    # Historical Laguna daily flows
    LDDFlow = np.loadtxt('./Input/LDxl_Catalog.txt', delimiter = ',')
    C = LDDFlow[:Fidx+1,:]
    LDDFlows = np.concatenate((C, LDDFlow), axis=0)
    LDDFlows[:Fidx+1,:3] = BTDFlows[:Fidx+1,:3]

    # Go through and sum the historical precip by water year
    for j in range(len(SC_Climate)):
        if SC_Climate[j,0] == 9:
            WaterYear = SC_Climate[j,1]
            idx = np.where(SC_Climate[:,1]==WaterYear)[0][0]
            idx1 = np.where(SC_Climate[:,1]==WaterYear)[0][-1]
            SC_Climate[idx:idx1+1,3] = np.sum(SC_Climate[idx:idx1+1,2])

    # Now do the same for the future climate precip time series
    Precip = np.c_[Precip, np.zeros((len(Precip),2))] # Add column of zeros to fill
    for j in range(HydroLength):
        if Precip[j,1] == 9:
            if j < 11:
                # Fill a water year array in the Precip variable
                Precip[:j+1,4] = Precip[j,0]
            else:
                # Fill a water year array in the Precip variable
                Precip[j-11:j+1,4] = Precip[j,0]
            WaterYear = Precip[j,4]
            idx = np.where(Precip[:,4]==WaterYear)[0][0]
            idx1 = np.where(Precip[:,4]==WaterYear)[0][-1]
            Precip[idx:idx1+1,5] = np.sum(Precip[idx:idx1+1,2])

    # Convert monthly rainfall to inches per month
    # This no longer is used in the historical calculation
    Precip[:,3] = Precip[:,2] / 10 / 2.54
    HC_Hgraph = np.ceil(HydroCond_Ann)
    HC_Hgraph = np.ones((len(BT_AnnFlow), 5))
    HC_Hgraph[:, 0] = BT_AnnFlow[:, 0]
    
    for j in range(HydroLength):
        if Precip[j,1] == 9:
            # Replace third column with precip total
            HC_Hgraph[int(j/12),2] = Precip[j,5]
            # This is the flag for the analog historical daily flow year to use
            HC_Hgraph[int(j/12),3] = AnalogYr[j]

    for j in range(len(HC_Hgraph)):
        WaterYear = HC_Hgraph[j,3]
        idx = np.where(SC_Climate[:,1] == WaterYear)[0][-1]
        HC_Hgraph[j,4] = SC_Climate[idx,3]

    dateLength = int(np.sum(BT_MonthlyFlow[:,0]))
    CC_ProjTimeSeries_NC = np.zeros((dateLength,8))

    # Date at start of time series
    A = np.datetime64(str(int(BT_MonthlyFlow[0, 2])) + '-' + str(int(BT_MonthlyFlow[0, 1])) + '-01')
    B = np.datetime64(str(int(BT_MonthlyFlow[-1, 2])) + '-0' + str(int(BT_MonthlyFlow[-1, 1])) + '-30')
    if ACOND == 1: ##### SAM: I think the else statement could be used in all cases
        ymd = pd.date_range(start = '1937-10-01', end = '2015-09-30')
    else:
        ymd = pd.date_range(start = A, end = B)
    y = np.array([val.year for val in ymd])
    m = np.array([val.month for val in ymd])
    d = np.array([val.day for val in ymd])
    CC_ProjTimeSeries_NC[:, 0] = m #ymd.astype('datetime64[M]').astype(int) % 12 + 1
    CC_ProjTimeSeries_NC[:, 1] = d #ymd.astype('datetime64[D]').astype(int) % 30 + 1
    CC_ProjTimeSeries_NC[:, 2] = y #ymd.astype('datetime64[Y]').astype(int) + 1970

    for j in range(dateLength):
        if CC_ProjTimeSeries_NC[j,0] >= 10:
            CC_ProjTimeSeries_NC[j,3] = CC_ProjTimeSeries_NC[j,2] + 1
        else:
            CC_ProjTimeSeries_NC[j,3] = CC_ProjTimeSeries_NC[j,2]
        b1 = str(int(CC_ProjTimeSeries_NC[j,2]))
        b2 = str(int(CC_ProjTimeSeries_NC[j,0]))
        str_val = b1 + b2
        CC_ProjTimeSeries_NC[j,4] = int(str_val)

    lambda_val = 250

    # Pass the BT flows to a looping variable
    DFlows = BTDFlows

    # Loop over all three northcoast streams
    for jjj in range(3):
        
        if jjj == 0:
            DFlows[:,4] = LGDFlows[:,3]
        elif jjj == 1:
            DFlows[:,4] = MJDFlows[:,3]
        else:
            DFlows[:,4] = LDDFlows[:,3]

        # Lets calculate the daily hydrographs
        for j in range(len(HC_Hgraph)):
            
            Ann_Precip = HC_Hgraph[j,2]
            WaterYear = HC_Hgraph[j,0]
            # Analog year
            AYear = HC_Hgraph[j,3]
            idx = np.where(CC_ProjTimeSeries_NC[:,3]==WaterYear)[0][0]
            idx1 = np.where(CC_ProjTimeSeries_NC[:,3]==WaterYear)[0][-1]
            diff1 = idx1-idx

            idx2 = np.where(DFlows[:,3]==AYear)[0][0]
            idx3 = np.where(DFlows[:,3]==AYear)[0][-1]
            diff2 = idx3-idx2

            if Ann_Precip == 0:
                for jj in range(idx, idx1+1):
                    # NOTE THIS ASSUMES THAT ALL RECORDS HAVE AT LEAST ONE WATER YEAR
                    # WITH PRECIPITATION
                    init_value = CC_ProjTimeSeries_NC[idx-1,5]
                    # Use the counter step down the exp function
                    decay_rate = np.exp(-(1/lambda_val) * ((jj + 1) - idx))
                    CC_ProjTimeSeries_NC[jj,5] = init_value * decay_rate
                    if CC_ProjTimeSeries_NC[jj,5] < 0:
                        CC_ProjTimeSeries_NC[jj,5] = 0
            else:
                # Just step through and project year by year since everything is in
                # sequential order
                if diff2 == 365 and diff1 == 364:
                    # Dealing with leap year mismatch here; Feb 29 is 152 days after
                    # start of the water year
                    CC_ProjTimeSeries_NC[idx:idx+151,5] = DFlows[idx2:idx2+151,4] * (HC_Hgraph[j,2]/HC_Hgraph[j,4])
                    CC_ProjTimeSeries_NC[idx+151:idx1+1,5] = DFlows[idx2+152:idx3+1,4] * (HC_Hgraph[j,2]/HC_Hgraph[j,4])
                elif diff2 == 364 and diff1 == 365:
                    # Dealing with leap year mismatch here; Feb 29 is 152 days after
                    # start of the water year
                    CC_ProjTimeSeries_NC[idx:idx+152,5] = DFlows[idx2:idx2+152,4] * (HC_Hgraph[j,2]/HC_Hgraph[j,4])
                    CC_ProjTimeSeries_NC[152,5] = CC_ProjTimeSeries_NC[151,5]
                    CC_ProjTimeSeries_NC[idx+152:idx1+1,5] = DFlows[idx2+151:idx3+1,4] * (HC_Hgraph[j,2]/HC_Hgraph[j,4])
                else:
                    CC_ProjTimeSeries_NC[idx:idx1+1,5] = DFlows[idx2:idx3+1,4] * (HC_Hgraph[j,2]/HC_Hgraph[j,4])

        [m, _] = CC_ProjTimeSeries_NC.shape

        # Correct the flow record at the end of each water year to provide a
        # smooth transition for year to year. Otherwise abrupt changes can occur.
        # Define a backward looking vector length to spread abrupt flow changes
        # This is for the first case < 1.15; i.e. decrease in flow at WY
        back_1 = 30
        # This is for the second case > 1.15; i.e. increase in flow at WY
        #back_2 = 122
        forward = 122
        forward_2 = 91
        efold_back = 1/15
        efold_forward = 1/500
        peak = 1.5

        for j in range(m):
            
            if j < m-1:
                
                if CC_ProjTimeSeries_NC[j,0] == 9 and CC_ProjTimeSeries_NC[j,1] == 30: #if Sept 30
                    
                    if (CC_ProjTimeSeries_NC[j,5] / CC_ProjTimeSeries_NC[j+1,5] > 1.150):
                        anchor_back = CC_ProjTimeSeries_NC[(j+1)-back_1,5]
                        delta1 = CC_ProjTimeSeries_NC[(j+1)-back_1,5]-CC_ProjTimeSeries_NC[(j+1),5]
                        array_back = np.arange(back_1, 0, -1)
                        
                        # for jj in range(back_1):
                        for jj in np.arange(back_1,0,-1):
                            CC_ProjTimeSeries_NC[(j+1)-jj,6] = anchor_back - (delta1 - (delta1 * np.exp(-efold_back*array_back[jj-1])))
                
                    if (CC_ProjTimeSeries_NC[j+1,5] / CC_ProjTimeSeries_NC[j,5] > 1.150):
                        
                        idx = np.where(CC_ProjTimeSeries_NC[j:(j+forward)+1,5] > peak)[0][0] #potentially -1
                        
                        if idx.size > 0:
                            
                            forward_find = j + (idx-2)
                            
                            for jj in range(j, forward_find+2): #used to be +1
                                
                                if jj == j + 1:
                                    CC_ProjTimeSeries_NC[jj,6] = CC_ProjTimeSeries_NC[jj-1,5] - (efold_forward * CC_ProjTimeSeries_NC[jj-1,5])
                                else:
                                    CC_ProjTimeSeries_NC[jj,6] = CC_ProjTimeSeries_NC[jj-1,6] - (efold_forward * CC_ProjTimeSeries_NC[jj-1,6])
                        
                        else:
    
                            # print('Error. No peak found.')
                            for jj in range(j, forward_2+1):
                                
                                if jj == j + 1:
                                    CC_ProjTimeSeries_NC[jj,6] = CC_ProjTimeSeries_NC[jj-1,5] - (efold_forward * CC_ProjTimeSeries_NC[jj-1,5])
                                else:
                                    CC_ProjTimeSeries_NC[jj,6] = CC_ProjTimeSeries_NC[jj-1,6] - (efold_forward * CC_ProjTimeSeries_NC[jj-1,6])
                
                else:
                
                    CC_ProjTimeSeries_NC[j,7] = 0

        for j in range(m):
            
            if CC_ProjTimeSeries_NC[j,6] == 0:
                
                CC_ProjTimeSeries_NC[j,6] = CC_ProjTimeSeries_NC[j,5]
                
            if CC_ProjTimeSeries_NC[j,0] == 2 and CC_ProjTimeSeries_NC[j,1] == 29: #if Feb 29
                
                CC_ProjTimeSeries_NC[j,6] = (CC_ProjTimeSeries_NC[j-1,5] + CC_ProjTimeSeries_NC[j+1,5])/2

        FinalFlow = np.zeros((m,4))
        FinalFlow[:,0] = CC_ProjTimeSeries_NC[:,0]
        FinalFlow[:,1] = CC_ProjTimeSeries_NC[:,1]
        FinalFlow[:,2] = CC_ProjTimeSeries_NC[:,2]
        FinalFlow[:,3] = CC_ProjTimeSeries_NC[:,6]

        if jjj == 0:
            LGc = FinalFlow
            for i in range(len(LGc)):
                if LGc[i,3] < 0.01:
                    LGc[i,3] = 0.01
        elif jjj == 1:
            MJc = FinalFlow
            for i in range(len(MJc)):
                if MJc[i,3] < 0.01:
                    MJc[i,3] = 0.01
        else:
            LDc = FinalFlow
            for i in range(len(LDc)):
                if LDc[i,3] < 0.01:
                    LDc[i,3] = 0.01

        CC_ProjTimeSeries_NC[:,5:] = 0
        DFlows[:,4] = 0

    return LGc, MJc, LDc


